In [13]:
# Import libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import  datetime

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

In [36]:
# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers=headers) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print (f'Recent News Headlines for {ticker} - {len(df_tr)}: ')
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL - 100: 
5G Stocks To Watch As Huawei Ramps Mate 60 Pro Vs. iPhone 15 ( Today 09:22AM )
This Apple Chip Supplier May Be Going Public. It Could Be a Mini Arm. ( 09:04AM )
This Apple Chip Supplier Is Going Public. It Could Be a Mini Arm. ( 09:04AM )


Recent News Headlines for TSLA - 99: 
Tesla slides as Goldman lowers profit forecasts amid price cuts, margin pressures ( Today 10:10AM )
UAW Strike Drives Wedge Into Fractured U.S. Auto Industry, Could Tesla Come Out On Top? ( 10:07AM )
Tesla price cuts to weigh on earnings: Goldman Sachs ( 10:06AM )


Recent News Headlines for AMZN - 100: 
The 2 Stocks Investors Must Watch This Week ( Today 09:21AM )
Here's How Much a $1000 Investment in Amazon Made 10 Years Ago Would Be Worth Today ( 08:30AM )
Hawaiian Airlines sees advantage in Amazon pivot to Airbus freighter ( 07:00AM )


In [37]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [38]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [39]:
news

,Ticker,Date,Time,Headline,neg,neu,pos,compound
0,AAPL,Today,09:22AM,5G Stocks To Watch As Huawei Ramps Mate 60 Pro...,0.000,1.000,0.000,0.0000
1,AAPL,Today,09:04AM,This Apple Chip Supplier May Be Going Public. ...,0.000,1.000,0.000,0.0000
2,AAPL,Today,09:04AM,This Apple Chip Supplier Is Going Public. It C...,0.000,1.000,0.000,0.0000
3,AAPL,Today,09:00AM,Apple AirPods Pro review: The best gets better,0.000,0.458,0.542,0.7964
4,AAPL,Today,08:59AM,Apple Stock Hasnt Had a New iPhone Boost. This...,0.000,0.597,0.403,0.6597
...,...,...,...,...,...,...,...,...
294,AMZN,Sep-12-23,01:53PM,Why United Parcel Service Stock Is Falling Today,0.154,0.577,0.269,0.2960
295,AMZN,Sep-12-23,12:39PM,Citi predicts positive growth for Microsoft af...,0.000,0.530,0.470,0.7351
296,AMZN,Sep-12-23,12:35PM,Amazon rolls out end-to-end global supply chai...,0.000,0.805,0.195,0.1779
297,AMZN,Sep-12-23,11:40AM,SNAP & MTV Team up to Revolutionize VMAs' Voti...,0.000,1.000,0.000,0.0000


In [41]:
# Nóticias de hoje
news['Date'] = news['Date'].replace('Today', datetime.date.today())
news['Date'] = pd.to_datetime(news['Date']).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe)
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')



              Date     Time  neg    neu    pos  compound
Ticker                                                  
AAPL    2023-09-18  09:22AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-18  09:04AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-18  09:04AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-18  09:00AM  0.0  0.458  0.542    0.7964
AAPL    2023-09-18  08:59AM  0.0  0.597  0.403    0.6597
...            ...      ...  ...    ...    ...       ...
AAPL    2023-09-14  04:28AM  0.0  0.741  0.259    0.2732
AAPL    2023-09-14  04:15AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-14  04:03AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-14  03:30AM  0.0  1.000  0.000    0.0000
AAPL    2023-09-14  02:24AM  0.0  1.000  0.000    0.0000

[100 rows x 6 columns]


              Date     Time    neg    neu    pos  compound
Ticker                                                    
TSLA    2023-09-18  10:10AM  0.355  0.473  0.172   -0.2732
TSLA    2023-09-18  10:07AM  0.092  0.798  0.110    0.